In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
pyspark

<module 'pyspark' from '/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/__init__.py'>

In [5]:
sc = pyspark.SparkContext()

In [6]:
from pyspark.sql import *

In [7]:
spark = SparkSession \
    .builder \
    .appName("Movie review") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [8]:
#dataframe = spark.read.csv('/home/cse587/train.csv', inferSchema=True)

In [9]:
df = spark.read.load("/home/cse587/train.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")

In [10]:
#df.printSchema()

In [11]:
df.columns
df.select("genre").show()

+--------------------+
|               genre|
+--------------------+
|['World cinema', ...|
|['Action/Adventur...|
|['Musical', 'Acti...|
|          ['Comedy']|
|['Crime Fiction',...|
|['Action/Adventur...|
|['Thriller', 'Dra...|
|           ['Drama']|
|['Black-and-white...|
|['Animation', 'Sh...|
|          ['Comedy']|
|['Crime Fiction',...|
|          ['Comedy']|
|          ['Comedy']|
|          ['Horror']|
|['Crime Fiction',...|
|           ['Drama']|
|['Crime Fiction',...|
|  ['Indie', 'Drama']|
|           ['Drama']|
+--------------------+
only showing top 20 rows



In [12]:
#text_split = text_file.flatMap(lambda line: line.split(" "))

#text_word = text_split.map(lambda word: (word,1))

#word_count = text_word.reduceByKey(lambda a,b: a + b)

In [13]:
newdf = df.union(df)
newdf.count()

62222

In [14]:
df.describe('movie_id').show()

+-------+--------------------+
|summary|            movie_id|
+-------+--------------------+
|  count|               31111|
|   mean|1.4278167534139128E7|
| stddev|1.0880709619946605E7|
|    min|            10000053|
|    max|             9999280|
+-------+--------------------+



In [15]:
#word_count.collect()

In [16]:
df.groupby('genre').count().show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|    ['Romance Film']|  362|
|['Crime Fiction',...|    1|
|['Crime Fiction',...|    1|
|['Adventure', 'Co...|    1|
|['Crime Fiction',...|    2|
|['Crime Fiction',...|    1|
|['Musical', 'Indi...|    1|
|['Action/Adventur...|    1|
|['World cinema', ...|    6|
|['Musical', 'Acti...|    2|
|['Romantic comedy...|    1|
|['Black-and-white...|    1|
|['Crime Fiction',...|    5|
|['Crime Fiction',...|    1|
|['Musical', 'Acti...|    1|
|['Thriller', 'Mys...|    1|
|['Action', 'Comed...|    1|
|['Action', 'Comed...|    2|
|['Comedy', 'Roman...|    1|
|['Action/Adventur...|    1|
+--------------------+-----+
only showing top 20 rows



In [17]:
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/cse587/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Relate movie_id and movie_name
movie_name = {}
labels = set()
for row in df.collect():
    if row['movie_id'] != None:
        movie_name[row['movie_id']] = row['movie_name']
    if row['genre'] != None:
        elements = row['genre'].split(',')
        for genre in elements:
            genre = genre.replace('[','')
            genre = genre.replace(']','')
            genre = genre.replace("\'",'')
            genre = genre.replace(" ",'')
            labels.add(genre)

    
labels

#print(len(labels))
#print(labels)
#print(movie_name)

{'Action',
 'Action/Adventure',
 'Adventure',
 'Animation',
 'Black-and-white',
 'Comedy',
 'CrimeFiction',
 'Drama',
 'FamilyFilm',
 'Horror',
 'Indie',
 'Musical',
 'Mystery',
 'RomanceFilm',
 'Romanticcomedy',
 'Romanticdrama',
 'ScienceFiction',
 'ShortFilm',
 'Thriller',
 'Worldcinema'}

In [19]:
from collections import defaultdict
word_frequency = defaultdict(int)
for row in df.collect():
    if row['plot'] != None:
        line = row['plot'].strip()
        line = line.lower()
        line = line.replace('_','')
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        words = tokenizer.tokenize(line)
        for word in words:
            word = re.sub(r'\d+', '', word)
            # Remove stopwords and invalid words. 
            if word not in stop_words and word != "":
                  word_frequency[word] = word_frequency[word] + 1
word_frequency = sorted(word_frequency.items(), key=lambda kv: kv[1], reverse = True)

word_frequency = [v[0] for v in word_frequency[2000:4000]]
key_words = set(word_frequency)
#print(key_words)

In [20]:
#print(labels)

In [21]:
label_set = [[0]* len(labels) for i in range(len(movie_name))]
labels_list = list(labels)
i = 0
for row in df.collect():
    if row['genre'] != None:
            elements = row['genre'].split(',')
            for genre in elements:
                genre = genre.replace('[','')
                genre = genre.replace(']','')
                genre = genre.replace("\'",'')
                genre = genre.replace(" ",'')
                for label in range (len(labels_list)):
                    if genre == labels_list[label]:
                        label_set[i][label] = 1
    i = i + 1

In [22]:
dataset = [[0]* len(key_words) for i in range(len(movie_name))]

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [24]:

i = 0
#print(len(movie_name))
for row in df.collect():
    j = 0
    line = row['plot']
    if line != None:
        line = line.lower()
        line = line.replace('_','')
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        words = tokenizer.tokenize(line)
        for word in key_words:
            if word in words:
                dataset[i][j] = words.count(word)
            j = j + 1
    i = i + 1

In [25]:
#word_count.saveAsTextFile('/home/cse587/wordcount')


In [26]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [27]:
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
X_train = np.array(dataset)
X_scaled = preprocessing.scale(X_train)
Y_train = np.array(label_set)

X_scaled, X_val, Y_train, Y_val = train_test_split(
X_scaled, Y_train, test_size=0.2)

24888
6223
24888
6223


In [28]:
clf.fit(X_scaled, Y_train)

/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [29]:
y_pred = clf.predict(X_val)

In [30]:
#y_pred[1]
#labels

In [38]:
def get_labels(label, y_pred):
    labels = []
    for i in range (len(y_pred)):
        if y_pred[i] == 1:
            labels.append(label[i])
    return labels
for i in range (len(y_pred)):
    print(i+1,get_labels(list(labels), y_pred[i]), get_labels(list(labels), Y_val[i]))

1 ['Drama'] ['Action', 'FamilyFilm', 'Comedy', 'Action/Adventure', 'Adventure']
2 ['Drama', 'Indie'] ['Drama']
3 ['Drama'] ['Drama', 'Worldcinema', 'Musical']
4 ['Drama', 'Adventure'] ['Comedy']
5 ['Drama', 'RomanceFilm', 'Musical', 'Black-and-white', 'Comedy'] ['RomanceFilm', 'Musical', 'Romanticcomedy']
6 ['Thriller', 'Drama', 'Musical', 'CrimeFiction', 'Mystery'] ['Thriller', 'Drama', 'CrimeFiction']
7 ['Animation', 'FamilyFilm', 'ScienceFiction'] ['Action', 'FamilyFilm']
8 ['Drama', 'RomanceFilm'] ['Drama', 'RomanceFilm', 'Romanticcomedy', 'Comedy']
9 [] ['Drama', 'FamilyFilm', 'Adventure']
10 [] ['Black-and-white', 'Comedy']
11 ['Horror', 'FamilyFilm', 'Comedy', 'ShortFilm'] ['Horror']
12 ['Drama'] ['Drama', 'Black-and-white']
13 ['Drama'] ['Worldcinema', 'Musical', 'Romanticdrama']
14 ['Thriller', 'Drama', 'Romanticcomedy'] ['Drama', 'Black-and-white']
15 ['Drama', 'RomanceFilm'] ['Drama', 'Worldcinema']
16 [] ['Action', 'Drama', 'Action/Adventure', 'Adventure']
17 ['Mystery'] ['

793 ['Drama'] ['RomanceFilm', 'FamilyFilm']
794 ['Drama'] ['FamilyFilm', 'Adventure']
795 ['Drama'] ['Animation', 'FamilyFilm']
796 [] ['Black-and-white']
797 ['Drama'] ['Black-and-white', 'Comedy']
798 ['Drama'] ['Action', 'Drama', 'RomanceFilm', 'Adventure']
799 ['Drama', 'Black-and-white'] ['Action', 'Drama']
800 ['Animation', 'FamilyFilm', 'ShortFilm'] ['Animation', 'Comedy', 'ShortFilm']
801 ['Drama', 'Black-and-white'] ['Drama']
802 ['ShortFilm'] ['ShortFilm']
803 ['Animation'] ['Animation']
804 ['Drama'] ['RomanceFilm', 'ShortFilm']
805 ['Drama', 'Worldcinema', 'CrimeFiction'] ['CrimeFiction', 'Adventure']
806 ['FamilyFilm', 'Comedy'] ['Animation', 'FamilyFilm', 'ShortFilm']
807 ['Thriller', 'Horror', 'Comedy'] ['Comedy']
808 [] ['Animation', 'FamilyFilm', 'ShortFilm']
809 ['Drama'] ['Drama', 'RomanceFilm']
810 ['Thriller', 'RomanceFilm', 'Horror', 'Black-and-white', 'Comedy'] ['Drama', 'Black-and-white']
811 ['Drama', 'RomanceFilm', 'Worldcinema', 'Romanticcomedy'] ['RomanceFil

1702 ['Action', 'Animation'] ['FamilyFilm', 'Comedy']
1703 [] ['RomanceFilm', 'Animation', 'FamilyFilm']
1704 ['Thriller', 'Action', 'CrimeFiction'] ['Thriller', 'Action', 'Drama', 'CrimeFiction']
1705 [] ['Thriller', 'Action', 'Action/Adventure']
1706 [] ['Musical', 'Comedy']
1707 ['Drama'] ['Drama']
1708 ['Comedy'] ['Musical', 'Animation', 'FamilyFilm', 'Comedy']
1709 ['Drama', 'Worldcinema', 'Romanticcomedy', 'CrimeFiction', 'Comedy'] ['Drama', 'Worldcinema', 'Musical']
1710 ['Animation'] ['Drama', 'Animation']
1711 ['Drama', 'Comedy'] ['Drama']
1712 ['Drama'] ['Drama', 'RomanceFilm', 'Musical', 'Mystery']
1713 ['Thriller', 'Mystery'] ['Thriller', 'Drama', 'CrimeFiction']
1714 ['Drama', 'Worldcinema', 'Comedy'] ['Thriller', 'Drama', 'Musical']
1715 ['Drama'] ['Drama', 'Indie', 'CrimeFiction']
1716 [] ['Thriller']
1717 ['Thriller', 'Comedy'] ['Drama']
1718 [] ['Action']
1719 ['Action', 'Drama', 'Action/Adventure'] ['Action/Adventure']
1720 ['Animation', 'ShortFilm'] ['Animation', 'Fa

3036 ['Thriller', 'Action', 'Worldcinema', 'Indie', 'FamilyFilm', 'CrimeFiction', 'Action/Adventure', 'Adventure'] ['Thriller', 'Action', 'Drama', 'CrimeFiction', 'Action/Adventure']
3037 ['Thriller', 'Horror'] ['Drama', 'RomanceFilm', 'CrimeFiction', 'Comedy']
3038 ['Thriller', 'CrimeFiction', 'Comedy'] ['Comedy']
3039 ['Drama'] ['Drama']
3040 ['Comedy'] ['Comedy']
3041 [] ['Thriller', 'Drama', 'Mystery']
3042 ['Mystery'] ['Thriller', 'Action', 'Comedy', 'Action/Adventure']
3043 ['Thriller', 'Horror', 'Mystery'] ['Thriller', 'CrimeFiction', 'Mystery']
3044 ['Drama', 'Comedy'] ['Drama', 'CrimeFiction']
3045 ['Drama', 'Horror', 'ScienceFiction'] ['Drama', 'Indie', 'Comedy']
3046 ['Comedy'] ['Comedy']
3047 ['Comedy'] ['Horror', 'Comedy']
3048 ['Comedy'] ['Drama', 'Comedy']
3049 ['Drama', 'ShortFilm'] ['Thriller']
3050 ['Drama'] ['Drama', 'RomanceFilm', 'Indie', 'Romanticdrama']
3051 [] ['Thriller', 'Drama', 'CrimeFiction', 'Mystery']
3052 ['Comedy'] ['ShortFilm']
3053 ['Drama'] ['Drama',

3967 ['Drama'] ['Drama', 'RomanceFilm', 'Indie', 'CrimeFiction', 'Romanticdrama']
3968 ['Thriller', 'Action/Adventure'] ['Horror']
3969 ['Drama', 'Romanticcomedy', 'Comedy'] ['RomanceFilm', 'FamilyFilm', 'Comedy']
3970 ['Drama', 'Indie', 'Black-and-white', 'Comedy'] ['Drama', 'CrimeFiction', 'Black-and-white', 'Comedy']
3971 ['Drama'] ['FamilyFilm']
3972 [] ['Horror']
3973 ['Drama', 'Worldcinema', 'CrimeFiction', 'Action/Adventure'] ['Action', 'Drama', 'Black-and-white', 'Action/Adventure']
3974 ['FamilyFilm', 'Adventure'] ['Drama']
3975 ['Thriller', 'Action', 'Drama', 'RomanceFilm', 'Indie', 'ScienceFiction', 'Romanticdrama', 'Comedy'] ['RomanceFilm', 'Romanticcomedy', 'Comedy']
3976 ['Animation'] ['Animation', 'ScienceFiction', 'Comedy']
3977 ['Animation', 'Comedy'] ['Animation', 'FamilyFilm']
3978 ['Drama', 'Animation', 'FamilyFilm'] ['Drama', 'Worldcinema', 'Black-and-white', 'Comedy']
3979 ['Thriller', 'CrimeFiction', 'Black-and-white', 'Action/Adventure', 'Adventure'] ['Drama', '

4869 [] ['Horror', 'ScienceFiction']
4870 ['Drama', 'Musical'] ['Drama', 'RomanceFilm', 'Romanticcomedy', 'Comedy']
4871 ['Drama'] ['Drama', 'CrimeFiction']
4872 ['CrimeFiction', 'Comedy', 'Action/Adventure'] ['Comedy']
4873 ['Drama'] ['Drama', 'Comedy']
4874 [] ['Drama']
4875 ['Drama', 'ScienceFiction'] ['Drama', 'RomanceFilm', 'Worldcinema', 'Musical', 'Romanticdrama', 'Comedy']
4876 ['Thriller'] ['Mystery', 'Comedy']
4877 ['Drama'] ['Action', 'Drama', 'RomanceFilm', 'Romanticdrama']
4878 ['Drama', 'RomanceFilm'] ['RomanceFilm', 'Romanticcomedy', 'Comedy']
4879 ['Thriller', 'Horror', 'Mystery'] ['Thriller', 'Horror', 'CrimeFiction', 'Mystery']
4880 ['Drama'] ['Drama']
4881 [] ['Drama']
4882 ['Drama'] ['Drama']
4883 ['Drama'] ['Action']
4884 ['Thriller', 'Drama', 'RomanceFilm'] ['Thriller', 'Action', 'Drama', 'RomanceFilm', 'CrimeFiction']
4885 ['Drama', 'Musical', 'ScienceFiction', 'Black-and-white'] ['Action', 'Adventure']
4886 [] ['Drama', 'Musical', 'Comedy']
4887 ['Animation', 'C

6202 ['Drama'] ['Thriller', 'Drama']
6203 ['Mystery'] ['Action', 'Drama']
6204 ['Action', 'Drama', 'ScienceFiction'] ['Drama', 'Worldcinema']
6205 ['Thriller', 'Romanticdrama'] ['Thriller', 'Action', 'CrimeFiction', 'Action/Adventure']
6206 ['Drama'] ['Action', 'Drama', 'Black-and-white', 'Adventure']
6207 ['Drama', 'RomanceFilm', 'Romanticcomedy', 'Black-and-white', 'Comedy'] ['Drama', 'RomanceFilm', 'Indie', 'Romanticcomedy', 'Comedy']
6208 ['Drama'] ['Drama']
6209 [] ['Horror']
6210 ['Horror', 'Comedy'] ['Drama', 'Worldcinema', 'Action/Adventure']
6211 ['FamilyFilm', 'Comedy'] ['Drama', 'RomanceFilm', 'Worldcinema', 'Romanticcomedy', 'Romanticdrama', 'Comedy']
6212 ['Drama', 'RomanceFilm', 'Indie', 'Comedy'] ['Drama', 'Worldcinema', 'Musical', 'FamilyFilm']
6213 ['Drama'] ['Drama', 'RomanceFilm', 'Worldcinema', 'Musical', 'Romanticdrama']
6214 [] ['Drama']
6215 ['Drama', 'Worldcinema'] ['CrimeFiction']
6216 ['Drama'] ['Drama']
6217 ['ShortFilm'] ['Animation', 'Comedy', 'ShortFilm']


In [39]:
print(Y_val[0])
print(y_pred[0])
f1_scores = f1_score(Y_val,y_pred, average='weighted')
f1_scores

[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


0.3793095204872345

In [40]:
import csv
# Relate movie_id and movie_name
df_test = spark.read.load("/home/cse587/test.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")
movie_name_test = []
for row in df_test.collect():
    if row['movie_id'] != None:
        movie_name_test.append(row['movie_id'])  

In [44]:

dataset_train = [[0]* len(key_words) for i in range(len(movie_name_test))]

In [45]:
i=0
for row in df_test.collect():
    j = 0
    line = row['plot']
    if line != None:
        line = line.lower()
        line = line.replace('_','')
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        words = tokenizer.tokenize(line)
        for word in key_words:
            if word in words:
                dataset_train[i][j] = words.count(word)
            j = j + 1
    i = i + 1
Y_train = np.array(dataset_train)

In [47]:
Y_scaled = preprocessing.scale(Y_train)
y_pred = clf.predict(Y_scaled)

In [64]:
with open('sample_part1.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['movie_id', 'predictions'])
    for i in range (len(y_pred)):
        temp=''
        for j in y_pred[i]:
            temp = temp + str(j) + ' '
        writer.writerow((movie_name_test[i],temp))

In [65]:
with open('mapping_part1.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    labels = list(labels)
    for i in range (len(labels)):
        writer.writerow((str(i),labels[i]))